# WebPie 

  *web services development framework for Python*
  
  https://webpie.github.io/

# History

* Started around 2005 as part of rewriting D0 Trigger Database suite
  * Migration from set of CGI/shell Python scripts to Apache/mod_python
  * Old code: mix of Python and HTML
  * Goals: 
    + Separate Python from HTML
      - Separate functionality from representation
    + Run under Apache/mod_python

* Since then was reused in multiple applications:
  * 2007-2008 - NIMI/Tissue migrated from Zope
  * 2010-2011 - ILC Ground Motion DB
  * 2011-...  - ECL suite (ShiftScheduler, Collaboration DB)
  * Query Engine
  * DES Spakers Bureau, Constants, Collaboration, Publications, Telemetry Viewer
  * IFBeam database
  * NOvA Hardware DB
  * Conditions databases - NOvA, Minerva, UConDB
  * FTS-Lite
  * CMS Rucio Consistency monitor - 2020-now
  * MetaCat - 2019-now
* Wide range of interactive web GUI applications and data web services

* Over the years was rewritten and renamed several times and became a stand-alone framework ``WebPie``

# Hello, World !

The simplest WebPie application

```python
from webpie import WPApp

def hello(request, relpath, **args):      # hello is a web method
    return "Hello, World !\n"             # always responds with "Hello, World !"

WPApp(hello).run_server(9000)             # create application and run it as HTTP server
```

```sh
$ curl http://localhost:9000
Hello, World !
```

# Web Method Arguments

```python
def hello(request, relpath, **args):      # <--- web method
    return "Hello, World !\n"             
```

* request - parsed HTTP request
* relpath - URL path relative to the web method
* optional keyword arguments - URL query string parsed

# Web Method Arguments

## request

```python
def hello(request, relpath, **args):      
    return "Hello, World !\n"             
```

``request`` is a WebOb Request object containing complete information about the HTTP request
* WebOb is a public domain library, part of Pylons project
* See http://www.webob.org/

# Web Method Arguments

## relpath

```python
def hello(request, relpath, **args):
    who = relpath or "World"
    return f"Hello, {who} !\n"             
```

``relpath`` is the URL path relative to the path to the web method

Example:

* URL: http://localhost:9000/webpie
* URL Path: ``/webpie`` = ``/`` + ``webpie``
  * ``/`` is path to web method. In our case, the ``hello`` method is at the top of the URL path
  * ``webpie`` is the path tail after the web method path = relative path
            


```sh
$ curl http://localhost:9000/webpie      # relpath is "webpie"
Hello, webpie !

$ curl http://localhost:9000/            # relpath is empty - use default
Hello, World !

$ curl http://localhost:9000/longer/multi/word/path     
Hello, longer/multi/word/path !
```

# Web Method Arguments

## keyword arguments

```python
def greeter(request, relpath, who="world"):      
    return f"Hello, {who} !\n"             
```

Named URL query arguments are passed to the web method as keyword arguments

Example:

* URL: http://localhost:9000/hello?who=webpie
* URL query string: ``who=webpie``
* Passed to the web method as 
    ```python
    response = webmethod(..., who="webpie")
    ```

```sh
$ curl http://localhost:9000/hello
Hello, world !

$ curl http://localhost:9000/hello?who=webpie
Hello, webpie !
```

By the way, in this example, what happens to ``hello`` ?
* it becomes ``relpath``, but it is ignored

# What does a web method return ?

Generally, a web method is supposed to return WebOb ``Response`` object
 * ``Request`` -> ``webmethod()`` -> ``Response``

As a *convenience*, WebPie lets you return some pieces of the response and then creates the ``Response`` object from them using defaults
  * str or bytes -> HTTP response with given body and 200 (success) status code
  * int -> HTTP response with given status code, empty body
  * iterable -> chunked response body, 200 status code
  * (int, str or bytes or iterable) -> status code, response body
  * (iterable, str) -> body, content type, 200 status code
  * (iterable, dictionary) -> body, headers
  * dictionary -> body=json.dumps(dictionary), content_type="text/json"
  * ...

In [ ]:
def method(self, request, relpath, **args):
    ...
    return "Hello, World !"                                    # simple text response
    return ["Hello", "World !"]                                # chunked text response
    return (format_csv(tup) for tup in my_data), "text/csv"    # CSV encoded data
    return {"name": "John Doe", "email":"jdoe@fnal.gov"}       # JSON enoded data
    return 403                                                 # standard "access denied" response

# Handlers

* Building an application with only one web method is quick and easy but limited
* What if we want to have more than one web method in the application ?
* Create a subclass of the WPHandler class and add multiple methods
* Pass the subclass (not a method) to the App constructor

In [ ]:
from webpie import WPApp, WPHandler
import time

class Handler(WPHandler):
    
    def hello(self, request, relpath, name="stranger"):      
        return f"Hello, {name}\n"
    
    def time(self, request, relpath):
        return time.ctime() + "\n"

WPApp(Handler).run_server(9000)          # create WPApp from the Handler class, not a function

```sh
$ curl http://localhost:9000/hello
Hello, stranger

$ curl http://localhost:9000/hello?name=world
Hello, world

$ curl http://localhost:9000/time
Fri Nov 12 15:16:07 2021
```

# URL Mapping with Handlers

```python
class Handler(WPHandler):
    
    def hello(self, request, relpath, name="stranger"):      
        return f"Hello, {name}\n"
    
    def time(self, request, relpath):
        return time.ctime() + "\n"

WPApp(Handler).run_server(9000)
```

Example URL: http://localhost:9000/hello?name=webpie

* URL path: ``/hello`` = ``/`` + ``hello`` 
    * ``/`` is path to the Handler (top)
    * ``hello`` is the name of the Handler's method
    * relpath is empty
* Query string: ``name=webpie``
    * keyword argument
      ```python
         handler.hello(..., name="webpie")
      ```


Example URL: http://localhost:9000/hi?name=webpie
* URL path: ``/hi`` = ``/`` + ``hi`` 
    * ``/`` is path to the Handler (top)
    * ``hi`` - *error: the Handler does not have method "hi"*


# Nested Handlers

* Handlers can be nested
* URL path *is* the path through the handlers/subhandlrers tree from the top to the web method
  - top handler -> subhandler -> ... subhandler -> web method
* Better, more manageable structure of the code, object oriented design

In [ ]:
class Clock(WPHandler):
    def time(self, request, relpath):                      # <- .../clock/time
        return time.ctime() + "\n"

class Greeter(WPHandler):
    def hello(self, request, relpath, name="stranger"):    # <- .../greet/hello 
        return f"Hello, {name}\n"
    
class RootHandler(WPHandler):
    def __init__(self, *params):
        WPHandler.__init__(self, *params)
        self.greet = Greeter(*params)             # <- http://host:port/greet/* 
        self.clock = Clock(*params)               # <- http://host:port/clock/*
    
WPApp(RootHandler).run_server(9000)

```sh
$ curl http://localhost:9000/clock/time
Fri Nov 12 15:30:56 2021

$ curl http://localhost:9000/greet/hello
Hello, stranger
```

# Nested Handlers - URL Mapping

```python
class Clock(WPHandler):
    def time(self, request, relpath):                      # <- .../clock/time
        return time.ctime() + "\n"

class Greeter(WPHandler):
    def hello(self, request, relpath, name="stranger"):    # <- .../greet/hello 
        return f"Hello, {name}\n"
    
class RootHandler(WPHandler):
    def __init__(self, *params):
        WPHandler.__init__(self, *params)
        self.greet = Greeter(*params)             # <- http://host:port/greet/* 
        self.clock = Clock(*params)               # <- http://host:port/clock/*
    
WPApp(RootHandler).run_server(9000)
```

Example URL: http://localhost:9000/greet/hello?name=webpie

* URL path: ``/greet/hello`` = ``/`` + ``greet`` + ``/`` + ``hello`` 
    * ``/`` is path to the RootHandler (top)
    * ``greet`` is path from the RootHandler to the Greeter
    * ``hello`` is the name of the Greeter's method
    * relpath is empty
* Query string: ``name=webpie``
    * keyword argument
      ```python
         handler.hello(..., name="webpie")
      ```

# App and Handler Lifetime

* WPApp object is *persistent*
  * Created once when the server process starts
  * Can be used to store state across requests

* WPHandler and Request objects are *transient*
  * Created and destroyed for each request
    * create Request
    * create Handler
    * process Request -> Handler -> Response
    * destroy Request, Handler
  * Handler has access to
    * WPApp object (-> *persistent state*)
    * Request object

In [ ]:
class MyHandler(WPHandler):
    
    def __init__(self, request, app):               # the Request and WPApp objects
        WPHandler.__init__(self, request, app)
        ...
        
    def method(self, request, relpath, *kw):        # same Request object
        ...
        app = self.App                              # self.App is the WPApp object

# Subclassing WPApp

* WPApp, as well as WPHandler, can be subclassed
  * In C++ terms, WPApp is a concrete virtual class
* Derived class handles app configuration, persistent context initialization, etc.
* Must call WPApp constructor

```python
class MyHandler(WPHandler):
    
    def __init__(self, request, app):
        WPHandler.__init__(self, request, app)
        ...

class MyApp(WPApp):

    def __init__(self, ...):
        WPApp.__init__(self, MyHandler)
        ...
        
MyApp(...).run_server(9000)
```

# Threaded Applications

* Middleware like uWSGI, Apache/mod_wsgi process HTTP requests on concurrent threads
* So does WebPie
* WebPie provides a mechanism to support inter-thread synchronization/locking using the persistent WPApp object
  * @app_synchronized decorator - coarse
  * WPApp object as a lock and locking context manager - more fine grain

# Threaded Applications

## WPApp object as a locking context manager

```python
# Simple dictionary as a web service

class Handler(WPHandler):
    
    def set(self, request, relpath, **kw):        # URL: .../set?name=value
        with self.App:                            # acquire App's RLock object
            for name, value in kw.items():
                self.App.Memory[name] = value     # record values into App memory
            return "OK\n"
    
    def get(self, request, varname):
        with self.App:                            # acquire App's RLock object
            value = self.App.Memory.get(varname)
            return value + "\n" if value is not None else 404

class App(WPApp):
    
    def __init__(self, *params, **args):
        WPApp.__init__(self, *params, **args)
        self.Memory = {}                          # accessible by concurrent threads
        
App(Handler).run_server(9000, max_connections=10) # up to 10 concurrent request threads
```

```sh
$ curl http://localhost:9000/get/x
# (not found, yet)

$ curl http://localhost:9000/set?x=2
OK

$ curl http://localhost:9000/get/x
2
```

# Threaded Applications

## @app_synchronized decorator

```python
@app_synchronized
def web_method(...):
    # code

# equivalent:
def web_method(...):
    with self.App:
        # code
```

In [ ]:
from webpie import WPApp, WPHandler, app_synchronized
class Handler(WPHandler):
    
    @app_synchronized
    def set(self, request, relpath, **kw):        # URL: .../set?name=value
        for name, value in kw.items():            
            self.App.Memory[name] = value         # record values in App memory
        return "OK\n"
    
    @app_synchronized
    def get(self, request, varname):
        value = self.App.Memory.get(varname)
        return value + "\n" if value is not None else 404

class App(WPApp):
    
    def __init__(self, *params, **args):
        WPApp.__init__(self, *params, **args)
        self.Memory = {}
        
App(Handler).run_server(9000, max_connections=10) # up to 10 concurrent request threads

# WSGI

* WSGI - Web Server Gateway Interface - standard for web server development in Python
  * https://www.python.org/dev/peps/pep-3333/
  * https://docs.python.org/3/library/wsgiref.html#module-wsgiref

* Apache/mod_wsgi, Nginx/uWSGI - available web servers where a WSGI application can be plugged

* WebPie
  * Any WebPie application (WPApp object) *is* a WSGI application
  * Can be plugged into mod_wsgi, uWSGI, etc.

# Running under uWSGI

```python
from webpie import WPApp, WPHandler
import time

class Handler(WPHandler):
    def hello(self, request, relpath, name="stranger"):      
        return f"Hello, {name}\n"
    def time(self, request, relpath):
        return time.ctime() + "\n"

# instead of App(Handler).run_server(9000) ...

application = WPApp(Handler)         # uWSGI, by default, expects "application"
```

```sh
$ uwsgi --wsgi-file wsgi_app.py --http :9000 
```

# Running Same Code Standalone and under uWSGI

```python
from webpie import WPApp, WPHandler
import time

class Handler(WPHandler):
    def hello(self, request, relpath, name="stranger"):      
        return f"Hello, {name}\n"
    def time(self, request, relpath):
        return time.ctime() + "\n"

application = WPApp(Handler)

if __name__ == "__main__":
    # running stand-alone, e.g.: "$ python wsgi_app.py 9000"
    import sys
    port = int(sys.argv[1])
    application.run_server(port)
else:
    # uWSGI knows to look for "application"
    pass
```

```sh
$ uwsgi --wsgi-file wsgi_app.py --http :9000 
$ python wsgi_app.py 9000
```

# WebPie HTTP Server

### Simple way:

```python
from webpie import WPApp, WPHandler

class MyApp(WPApp):
    ...
        
MyApp(...).run_server(9000)
```

### Shortcut for:

```python
from webpie import WPApp, WPHandler, HTTPServer
...

class MyApp(WPApp):
    ...
        
application = MyApp(...)
server = HTTPServer(9000, application)
server.run()
```

# WebPie HTTP Server

### Threaded application:

```python
from webpie import WPApp, WPHandler, HTTPServer
...

class MyApp(WPApp):
    ...
        
application = MyApp(...)
server = HTTPServer(9000, application, max_connections=5, max_queued=20)     # HTTPSever is a Thread
server.start()                                
server.join()
```

* Optional ``max_connections``, ``max_queued`` arguments control thread concurrency

* ``application`` in fact can be any WSGI application, not necessarily WPApp

# Other Features

* Support for sessions - use WPSessionApp subclass of WPApp
  * uses cookies to carry session id
  * persistent session data storage
* Static file server - WPStaticHandler (WPHandler subclass) - pluggable into the Handler tree
  * Add static content (images, icons, JavaScript, CSS, etc.) to your application
* Method restrictions - strict applications
  * not all methods of Handler class need to be exposed as web methods
  * simple: methods starting with "_" are not web methods
  * alternative: @webmethod decorator
* HTTPS support
* Jinja2 support - if jinja2 is installed
  * Conveniently render the template and wrap it into the Response object
* Built in HTTP/HTTPS server
  * multi-process
  * multi-threaded
  * multi-app - route by URL path prefix


# Installation

```sh
$ pip install webpie
```
or
```sh
$ pip install --user webpie
```

# GitHub

https://github.com/webpie/webpie.git